## In this notebook we convert the data to a processed format to be able to better work with the data.

In [ ]:
%matplotlib inline 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import dates as mdates
import collections
import os

In [ ]:
processed_folder_path = os.path.join("..", "data", "processed")
unprocessed_folder_path = os.path.join("..", "data", "unprocessed")

In [ ]:
def pandas_eda(df):
    print("First 5 columns")
    print(df.head())
    print("*" * 100)
    print("Dataframe information")
    print(df.info())
    print("*" * 100)
    print("Missing values")
    print(df.isnull().sum())
    print("*" * 100)

### ENSO Index

We include the ENSO index as a column denoting a wet year, dry year or normal year.

$
flag = \begin{cases}
    -1, \text{if } \text{ ENSO } < -0.5 \\
    0, \text{if } 0.5 < \text{ ENSO } < 0.5 \\
    1, \text{if } \text{ ENSO } > 0.5
\end{cases}
$

with -1 denoting a dry year, 0 denoting a normal year and 1 denoting a wet year.

In [ ]:
df = pd.read_excel(os.path.join(unprocessed_folder_path, "Data_ENSO", "ENSO evolution.xlsx"), skiprows=2)

# df["avg"] = df.loc[:, df.columns != "Year"].mean(axis=1)

year_data = collections.defaultdict(list)

for index, row in df.iterrows():
    year = row["Year"]
    # Add only months that are in the corresponding hydrological year
    year_data[year - 1].extend(row[1:4].values)
    year_data[year].extend(row[4:].values)

year_indices = {}

# Here we also consider 1949 and 2021, even though we don't 
# have full data for those hydrological years yet
for year, enso_indices in year_data.items():
    avg_enso_index = np.mean(enso_indices)

    # Determine "wetness" of year
    if avg_enso_index < -0.5:
        index = -1
    elif -0.5 < avg_enso_index < 0.5:
        index = 0
    elif avg_enso_index > 0.5:
        index = 1

    year_indices[year] = index

print(year_indices)

#### Here I plot some information about the distribution of years

In [ ]:
counter = collections.Counter(year_indices.values())
labels, values = zip(*counter.items())

indexes = np.arange(len(labels))
width = 0.5

plt.bar(indexes, values, width, align="edge") 
plt.title("Number of dry (-1) years, normal (0) years and wet (1) years")
plt.xticks(indexes + width * 0.5, labels)
plt.show()

plt.figure()
plt.plot(year_indices.keys(), year_indices.values(), 'ob:')
plt.title("Distribution of dry (-1) years, normal (0) years and wet (1) years")
plt.show()

### Convert data files from NDSI/NDVI to formatted CSVs

In [ ]:
data_folder = "Data_NDSI_NDVI"
data_files = ["NDSI.txt", "NDVI.txt"]

for data_file in data_files:
    df = pd.read_csv(os.path.join(unprocessed_folder_path, data_folder, data_file), 
                     delimiter="\t", index_col=False,
                     names=["Watershed", "Subsubwatershed", "Product", "Date",
                            "Areaini", "Areareproj", f"Surfmax", f"Surfmin",
                            "Surfavg", "max", "min", f"avg",
                            "Surfcloudmax", "Surfcloudmin", "Surfcloudavg",
                            "Surfbadpixmax", "Surfbadpixmin", "Surfbadpixavg"])
    
    # Convert date to datetime and convert to wateryears
    df["date"] = pd.to_datetime(df["Date"])
    df = df.drop("Date", axis=1)
    
    df.to_csv(os.path.join(processed_folder_path, data_file[:4] + ".csv"))

In [ ]:
print(df.columns)

### Convert data from the river flow dataset to properly formatted CSV

In [ ]:
flow_data_folder = "Data_RiverFlow"
flow_data_file = "DGA.txt"

date_columns = ["day", "month", "year", "hour"]

df = pd.read_csv(os.path.join(unprocessed_folder_path, flow_data_folder, flow_data_file), 
                 delimiter="\t", index_col=False,
                 names=["station_number", "day", "month", "year", "hour", 
                        "river_height", "river_flow", "information", "origin"])

# Convert date go datetime and add as column
date = pd.to_datetime(dict(year=df.year, month=df.month, day=df.day, hour=df.hour))
df = df.drop(columns=date_columns)
df.insert(1, 'date', date)
df["water_year"] = df["date"].dt.year.where(df["date"].dt.month > 3, df["date"].dt.year - 1)


# Add year_type column, determining wetness of year
df["year_type"] = df['water_year'].map(year_indices)

df.to_csv(os.path.join(processed_folder_path, flow_data_file[:3] + ".csv"))